<a href="https://colab.research.google.com/github/uroiff/ffmpegConcatAudio/blob/main/FFmpeg-in-Google-Drive-Concat-Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/1bd5d8aeb88c8f402a9aea4846fab6304e7d8e43/Google-Drive-Logo.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>

from google.colab import drive

mode = "Mount" #@param ["Mount", "Unmount"]
drive.mount._DEBUG = False

if mode == "Mount":
    drive.mount('/content/drive', force_remount=True)
elif mode == "Unmount":
    try: drive.flush_and_unmount()
    except ValueError: pass
    !rm -rf /root/.config/Google/DriveFS

# __Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/32abf44ff4c8d145a94a24611f01141926a8daaa/FFmpeg-Logo.svg' height="40" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

## __Concatenate Audio Files__

In [ ]:
#@markdown <h3><b>Concatenate Audio Files to MP3</h3>
import os, sys, re
from pathlib import Path
import subprocess
from google.colab import drive

# First, mount Google Drive if not already mounted
try:
    drive_already_mounted = os.path.exists('/content/drive')
    if not drive_already_mounted:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully")
    else:
        print("Google Drive is already mounted")
except:
    print("Error mounting Google Drive. Please run drive.mount('/content/drive') manually")

# Input and output folders - using exact path format
input_folder = '/content/drive/MyDrive/YouTube/CoNhan Audio/1' #@param {type:"string"}
output_folder = input_folder #@param {type:"string"}
txt_folder = input_folder #@param {type:"string"}

#@markdown <h3><b>Output Settings:</h3>
output_filename = 'concatenated_output.mp3' #@param {type:"string"}
mp3_bitrate = '192k' #@param {type:"string"}

# Create output and txt folders if they don't exist
Path(output_folder).mkdir(parents=True, exist_ok=True)
Path(txt_folder).mkdir(parents=True, exist_ok=True)

# Function to extract the second number from filename for sorting
def extract_second_number(filename):
    # Extract all numbers from the filename
    numbers = re.findall(r'\d+', filename)
    if len(numbers) >= 2:
        # If there are at least two numbers, use the second one for sorting
        return int(numbers[1])
    elif len(numbers) == 1:
        # If there's only one number, use that
        return int(numbers[0])
    # If no numbers, return the filename as is
    return filename

# Debug: Print the exact files in the directory
print(f"Checking directory: {input_folder}")
try:
    # List all items in the directory
    items = os.listdir(input_folder)
    print(f"Total items found in directory: {len(items)}")

    # Filter just the files (not directories)
    all_files = [f for f in items if os.path.isfile(os.path.join(input_folder, f))]

    # Sort files by the second numeric value in each filename
    all_files.sort(key=extract_second_number)

    print(f"Found {len(all_files)} files sorted by second number:")
    for file in all_files:
        print(f"  - {file}")

except Exception as e:
    print(f"Error accessing directory: {e}")
    sys.exit(1)

if not all_files:
    print("No files found in the input folder.")
else:
    # Create a temporary text file for FFmpeg concat demuxer
    concat_list_path = os.path.join(output_folder, 'concat_list.txt')
    with open(concat_list_path, 'w') as f:
        for file in all_files:
            full_path = os.path.join(input_folder, file)
            f.write(f"file '{full_path}'\n")

    # FFmpeg command to concatenate files and encode to MP3
    output_path = os.path.join(output_folder, output_filename)
    txt_output_path = os.path.join(txt_folder, f"{output_filename}.txt")

    print(f"Concatenating {len(all_files)} files into {output_path}...")
    cmd = f'ffmpeg -v error -stats -f concat -safe 0 -i "{concat_list_path}" -c:a mp3 -b:a {mp3_bitrate} "{output_path}" 2>"{txt_output_path}"'

    # Execute the command
    try:
        print(f"Executing command: {cmd}")
        result = subprocess.run(cmd, shell=True, check=True, text=True)
        print(f"Successfully concatenated files into {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error during concatenation: {e}")
        print(f"Check the log file for details: {txt_output_path}")

    # Clean up the temporary file
    os.remove(concat_list_path)
    print("Temporary files cleaned up.")